# FP-Growth

The FP-growth algorithm is described in the paper Han et al., Mining frequent patterns without candidate generation, where “FP” stands for frequent pattern. Given a dataset of transactions, the first step of FP-growth is to calculate item frequencies and identify frequent items. Different from Apriori-like algorithms designed for the same purpose, the second step of FP-growth uses a suffix tree (FP-tree) structure to encode transactions without generating candidate sets explicitly, which are usually expensive to generate. After the second step, the frequent itemsets can be extracted from the FP-tree. In spark.mllib, we implemented a parallel version of FP-growth called PFP, as described in Li et al., PFP: Parallel FP-growth for query recommendation. PFP distributes the work of growing FP-trees based on the suffixes of transactions, and hence is more scalable than a single-machine implementation. We refer users to the papers for more details.

`spark.ml`’s FP-growth implementation takes the following (hyper-)parameters:

- `minSupport`: the minimum support for an itemset to be identified as frequent. For example, if an item appears 3 out of 5 transactions, it has a support of 3/5=0.6.

- `minConfidence`: minimum confidence for generating Association Rule. Confidence is an indication of how often an association rule has been found to be true. For example, if in the transactions itemset X appears 4 times, X and Y co-occur only 2 times, the confidence for the rule X => Y is then 2/4 = 0.5. The parameter will not affect the mining for frequent itemsets, but specify the minimum confidence for generating association rules from frequent itemsets.

- `numPartitions`: the number of partitions used to distribute the work. By default the param is not set, and number of partitions of the input dataset is used.

The `FPGrowthModel` provides:

- `freqItemsets`: frequent itemsets in the format of DataFrame(“items”[Array], “freq”[Long])
- `associationRules`: association rules generated with confidence above minConfidence, in the format of DataFrame(“antecedent”[Array], “consequent”[Array], “confidence”[Double]).
- `transform`: For each transaction in itemsCol, the transform method will compare its items against the antecedents of each association rule. If the record contains all the antecedents of a specific association rule, the rule will be considered as applicable and its consequents will be added to the prediction result. The transform method will summarize the consequents from all the applicable rules as prediction. The prediction column has the same data type as itemsCol and does not contain existing items in the itemsCol.

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth

spark = SparkSession.builder.master("local[1]").appName("FP-Growth").getOrCreate()

In [3]:
df = spark.createDataFrame([
    (0, [1, 2, 5]),
    (1, [1, 2, 3, 5]),
    (2, [1, 2])
], ["id", "items"])

In [4]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(df)

In [6]:
# Display frequent itemsets.
model.freqItemsets.show()

+---------+----+
|    items|freq|
+---------+----+
|      [5]|   2|
|   [5, 2]|   2|
|[5, 2, 1]|   2|
|   [5, 1]|   2|
|      [2]|   3|
|   [2, 1]|   3|
|      [1]|   3|
+---------+----+



In [7]:
# Display generated association rules.
model.associationRules.show()

+----------+----------+------------------+----+------------------+
|antecedent|consequent|        confidence|lift|           support|
+----------+----------+------------------+----+------------------+
|    [5, 2]|       [1]|               1.0| 1.0|0.6666666666666666|
|    [5, 1]|       [2]|               1.0| 1.0|0.6666666666666666|
|       [5]|       [2]|               1.0| 1.0|0.6666666666666666|
|       [5]|       [1]|               1.0| 1.0|0.6666666666666666|
|       [2]|       [5]|0.6666666666666666| 1.0|0.6666666666666666|
|       [2]|       [1]|               1.0| 1.0|               1.0|
|       [1]|       [5]|0.6666666666666666| 1.0|0.6666666666666666|
|       [1]|       [2]|               1.0| 1.0|               1.0|
|    [2, 1]|       [5]|0.6666666666666666| 1.0|0.6666666666666666|
+----------+----------+------------------+----+------------------+



In [8]:
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df).show()

+---+------------+----------+
| id|       items|prediction|
+---+------------+----------+
|  0|   [1, 2, 5]|        []|
|  1|[1, 2, 3, 5]|        []|
|  2|      [1, 2]|       [5]|
+---+------------+----------+



In [9]:
spark.stop()

## Example Task

Next, you can play with the given dataset `example_dataset.csv`, it will be downloaded into the current directory. 

In [10]:
!wget -O example_dataset.csv 'https://raw.githubusercontent.com/tnurbek/ds702/main/Lab14/products.csv'

--2022-04-20 20:21:27--  https://raw.githubusercontent.com/tnurbek/ds702/main/Lab14/products.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 494162 (483K) [text/plain]
Saving to: ‘example_dataset.csv’

example_dataset.csv 100%[===================>] 482.58K  --.-KB/s    in 0.02s   

2022-04-20 20:21:28 (20.7 MB/s) - ‘example_dataset.csv’ saved [494162/494162]



Create session/context...

In [11]:
# code here
spark = SparkSession.builder.master("local[1]").appName("FP-Growth").getOrCreate()
sc = spark.sparkContext

Use this function (`getFromFile()`) in order to read your dataset as needed, appending all items into a list. 

In [12]:
from csv import reader

def getFromFile(filename):
    itemSetList = []
    
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for line in csv_reader:
            line = list(filter(None, line))
            itemSetList.append(line)

    return itemSetList

In [13]:
itemSetList = getFromFile('example_dataset.csv')

In [14]:
itemSetList[:5]

[['Lassi', 'Coffee Powder', 'Butter', 'Yougurt', 'Ghee', 'Cheese'],
 ['Ghee', 'Coffee Powder'],
 ['Lassi', 'Tea Powder', 'Butter', 'Cheese'],
 ['Cheese', 'Tea Powder', 'Panner', 'Coffee Powder', 'Butter', 'Bread'],
 ['Cheese', 'Yougurt', 'Coffee Powder', 'Sugar', 'Butter', 'Sweet']]

Make your dataframe look like: 
``` 
[
    (0, [1, 2, 5]),
    (1, [1, 2, 3, 5]),
    (2, [1, 2])
]
```

In [15]:
# code here
lst = list(map(lambda x, y: (x, y), range(len(itemSetList)), itemSetList))

In [16]:
lst[:5]

[(0, ['Lassi', 'Coffee Powder', 'Butter', 'Yougurt', 'Ghee', 'Cheese']),
 (1, ['Ghee', 'Coffee Powder']),
 (2, ['Lassi', 'Tea Powder', 'Butter', 'Cheese']),
 (3, ['Cheese', 'Tea Powder', 'Panner', 'Coffee Powder', 'Butter', 'Bread']),
 (4, ['Cheese', 'Yougurt', 'Coffee Powder', 'Sugar', 'Butter', 'Sweet'])]

Create dataframe called `df`.

In [17]:
# code here
df = spark.createDataFrame(lst, ["id", "items"])

In [18]:
df.take(5)

[Row(id=0, items=['Lassi', 'Coffee Powder', 'Butter', 'Yougurt', 'Ghee', 'Cheese']),
 Row(id=1, items=['Ghee', 'Coffee Powder']),
 Row(id=2, items=['Lassi', 'Tea Powder', 'Butter', 'Cheese']),
 Row(id=3, items=['Cheese', 'Tea Powder', 'Panner', 'Coffee Powder', 'Butter', 'Bread']),
 Row(id=4, items=['Cheese', 'Yougurt', 'Coffee Powder', 'Sugar', 'Butter', 'Sweet'])]

Next, try to do the same approach as above, create a `FPGrowth` model, and find frequent itemsets, you may use different hyperparameters. 

In [19]:
# code here
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.2, minConfidence=0.15)
model = fpGrowth.fit(df)

In [20]:
df.count()

12526

In [21]:
model.freqItemsets.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|            [Cheese]|5476|
|[Cheese, Coffee P...|2517|
|     [Cheese, Bread]|2530|
|   [Cheese, Yougurt]|2532|
|        [Tea Powder]|5383|
|     [Coffee Powder]|5509|
|[Coffee Powder, G...|2578|
|[Coffee Powder, M...|2518|
|             [Sweet]|5483|
|      [Sweet, Bread]|2539|
|       [Sweet, Milk]|2512|
|             [Bread]|5484|
|[Bread, Coffee Po...|2528|
|    [Bread, Yougurt]|2507|
|       [Bread, Milk]|2517|
|            [Butter]|5481|
|     [Butter, Sweet]|2543|
|      [Butter, Ghee]|2530|
|   [Butter, Yougurt]|2529|
|     [Butter, Sugar]|2571|
+--------------------+----+
only showing top 20 rows



In [22]:
model.associationRules.show()

+---------------+---------------+-------------------+------------------+-------------------+
|     antecedent|     consequent|         confidence|              lift|            support|
+---------------+---------------+-------------------+------------------+-------------------+
|      [Yougurt]|       [Cheese]|0.46011266581864435| 1.052478314836439|  0.202139549736548|
|      [Yougurt]|        [Bread]| 0.4555696892604034|1.0405663617206078|0.20014370110170845|
|      [Yougurt]|       [Butter]|0.45956750863165546|1.0502723249626194|0.20190004790036722|
|      [Yougurt]|[Coffee Powder]|0.46429220425222606|1.0556769196702456|0.20397573048060036|
|      [Yougurt]|         [Milk]|0.45666000363438125|1.0351290636127868|0.20062270477406993|
|      [Yougurt]|        [Sugar]|0.45956750863165546|1.0500807393506233|0.20190004790036722|
|       [Cheese]|[Coffee Powder]| 0.4596420745069394|1.0451037620754988|0.20094204055564427|
|       [Cheese]|        [Bread]| 0.4620160701241782|1.055290535079404

In [23]:
model.transform(df).show(truncate=50)

+---+--------------------------------------------------+--------------------------------------------------+
| id|                                             items|                                        prediction|
+---+--------------------------------------------------+--------------------------------------------------+
|  0|[Lassi, Coffee Powder, Butter, Yougurt, Ghee, C...|               [Bread, Milk, Sugar, Sweet, Panner]|
|  1|                             [Ghee, Coffee Powder]|[Butter, Lassi, Milk, Panner, Sugar, Cheese, Br...|
|  2|               [Lassi, Tea Powder, Butter, Cheese]|[Coffee Powder, Bread, Yougurt, Sweet, Ghee, Su...|
|  3|[Cheese, Tea Powder, Panner, Coffee Powder, But...|        [Yougurt, Sweet, Ghee, Sugar, Milk, Lassi]|
|  4|[Cheese, Yougurt, Coffee Powder, Sugar, Butter,...|                        [Bread, Milk, Ghee, Lassi]|
|  5|    [Sugar, Tea Powder, Ghee, Sweet, Panner, Milk]|    [Coffee Powder, Butter, Lassi, Bread, Yougurt]|
|  6|                       